### Sumary final canopy model

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,Lasso
from sklearn.cluster import KMeans
from sklearn import svm
import statsmodels.api as sm
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
import plotly.plotly as py
import pandas as pd



%matplotlib inline

In [2]:
df_Capital = pd.read_csv('CCapital_data.txt',delimiter="\t",error_bad_lines=False)
df_Counties = pd.read_csv('US_Counties.txt', delimiter="\t")
df_Capital.fillna(value=0, inplace=True)
df_Capital.drop(['Unnamed: 7'], axis=1,inplace=True)

df_BEA = pd.read_csv('BEA_income.txt', delimiter="\t")
df_Unemp = pd.read_csv('unemployment.txt', delimiter="\t")
df_BEA['fips'] = df_BEA['fips'].astype(int)
df_Unemp.drop(df_Unemp[['Code','Code.1','Code.2','Unnamed: 5']],axis=1, inplace=True)


df_Counties['Longitude'] = df_Counties['Longitude'].astype(float)
df_Counties['Latitude'] = df_Counties['Latitude'].astype(float)
df_Counties['Population'] = df_Counties['Population'].astype(float)
df_Counties['Land_Miles'] = df_Counties['Land_Miles'].astype(float)
df_Counties['Water_miles'] = df_Counties['Water_miles'].astype(float)

df_Counties['fips'] = df_Counties['fips'].astype(int)
#df_Capital[' County Geo ID'] = df_Capital[' County Geo ID'].astype(int)
df_Capital.columns = ['Amount', 'County', 'State', 'Investment Type',
       'Organization Type', 'Mission-Related Investment', 'fips']

df_feature = pd.merge(df_BEA, df_Unemp[['Rate','fips','Force','Employed','Unemployed']], \
                      how='inner', on=[ 'fips','fips']).copy()
df_feature = pd.merge(df_Counties,df_feature,how='inner', on=[ 'fips','fips']).copy()
df_feature['ldensity'] =  df_feature['Land_Miles']/df_feature['Population']

df_Final = pd.merge(df_Capital, df_feature, how='inner', on=[ 'fips','fips'])

df_Model = pd.DataFrame(df_Final[['fips','Population','Land_Miles','Water_miles','Latitude','Longitude','Rate',\
                                  'Employed','Unemployed','2013','D_2014','RR_2014','P_2013','P_2014','R_2014','ldensity']])
df_Model['label'] = 1

df_Model0  = pd.DataFrame(df_feature[~df_feature['fips'].isin(df_Model['fips']) & ((df_feature['State_x']== 'WA') | \
                (df_feature['State_x']== 'OR') | (df_feature['State_x']== 'AK')|(df_feature['State_x']== 'MT')\
                 | (df_feature['State_x']== 'WY')   |(df_feature['State_x']== 'ID')  )])
df_Model0 = pd.DataFrame(df_Model0[['fips','Population','Land_Miles','Water_miles','Latitude','Longitude','Rate',\
                                  'Employed','Unemployed','2013','D_2014','RR_2014','P_2013','P_2014','R_2014','ldensity']])
df_Model0['label'] = 0

df_Final = pd.DataFrame(df_Model.append(df_Model0).drop_duplicates())
df_Final['fips'] = df_Final['fips'].astype(int)

if 'fips' != df_Final.index.name:
    df_Final.set_index('fips',inplace =True)
    
if 'label' in df_Final.columns:
    df_y = df_Final.pop('label')

Skipping line 172: expected 8 fields, saw 9
Skipping line 684: expected 8 fields, saw 10



In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_Final, df_y,test_size=0.25, random_state=2)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
#print "Accuracy on test set:", rf.score(X_test, y_test)

logistic_l2 = LogisticRegression(C=100, penalty='l2', tol=0.001)
logistic_l2.fit(X_train, y_train)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False)

In [12]:
#print "Accuracy on test set:", logistic_l2.score(X_test, y_test)

### Get NW Prababilities

In [14]:
prob = pd.DataFrame(logistic_l2.predict_proba(df_Final))
prob.columns = ['prob0','prob1']

prob = prob.round(4)

df_map = df_Final.copy()
df_map.reset_index(inplace=True)
df_map  = pd.concat([df_map,prob],axis=1).copy()
df_map = pd.merge(df_feature[['State_y','County','fips']],df_map,on=[ 'fips','fips'])

df_map['text'] = df_map['County']+' : '+df_map['State_y']+'<br> Population : ' + \
                df_map['Population'].astype(str)+ '<br> PerCapita Income : ' + df_map['D_2014'].astype(str) +\
                '<br> Unemployment Rate : ' + df_map['Rate'].astype(str)

### Get US Probabilities

In [16]:
df_fforProb = df_feature[['fips','Population','Land_Miles','Water_miles','Latitude','Longitude','Rate',\
                                  'Employed','Unemployed','2013','D_2014','RR_2014','P_2013','P_2014','R_2014','ldensity']].copy()
if 'fips' != df_fforProb.index.name:
    df_fforProb.set_index('fips',inplace =True)

prob_us = pd.DataFrame(logistic_l2.predict_proba(df_fforProb))
prob_us.columns = ['prob0','prob1']
prob_us = prob_us.round(4)

df_map_us = df_fforProb.copy()
df_map_us.reset_index(inplace=True)
df_map_us  = pd.concat([df_map_us,prob_us],axis=1).copy()
df_map_us = pd.merge(df_feature[['State_x','Counties','fips']],df_map_us,on=[ 'fips','fips'])

df_map_us['text'] = df_map_us['Counties']+ ' : ' + df_map_us['State_x']+'<br> Population : ' +\
                 df_map_us['Population'].astype(str)+ '<br> Percapita Income : ' + df_map_us['D_2014'].astype(str) + \
                '<br> Unemp Rate : ' + df_map_us['Rate'].astype(str)

In [22]:
#df_map_us['text'] = df_map_us['text'].map(lambda x: unicode(x, errors='ignore'))


### Plot US Probabilities

In [23]:


scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_map['Longitude'],
        lat = df_map['Latitude'],
        text = df_map['text'],
        mode = 'markers',
        marker = dict( 
            size = 8, 
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = df_map_us['prob0'],
            cmax = df_map_us['prob0'].max(),
            colorbar=dict(
                title="map showing probability areas"
            )
        ))]

layout = dict(
        title = ' Least amount of contributions ',
        colorbar = True,   
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename=' Contributions by County' )



### map all geos

In [24]:
final_map = pd.merge(df_Capital[['Organization Type','State','Investment Type','Amount','fips']],\
                 df_map_us,how='right',on=[ 'fips','fips']).copy()


final_map = pd.merge(df_Capital[['Organization Type','State','Investment Type','Amount','fips']],\
                 df_map,how='right',on=[ 'fips','fips']).copy()
final_map.fillna('No Orgs/Investments Present',inplace=True)

cases = []
#colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)','rgb(239,243,255)']
colors = ["rgb(0,116,217)","lightyellow","rgb(255,65,54)","rgb(225,35,204)","rgb(133,20,75)","rgb(255,133,27)","lightgrey", \
         'rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)',"lightpink","lightgreen"]

scl = [ [0,"lightyellow"],[0.35,"lightgrey"],[0.5,"lightpink"],\
    [0.6,"lightgreen"],[0.7,"lightblue"],[1,"rgb(33,113,181)"] ]

scl = [ [0,"lightyellow"],[0.35,"lightgrey"],[0.5,"lightpink"],\
    [0.6,"lightgreen"],[0.7,"lightblue"],[1,"rgb(33,113,181)"] ]

limits = final_map['Organization Type'].unique()
final_map['text'] =  final_map['County']+ ' : ' + final_map['State_y']+'<br> Population : ' +\
                 final_map['Population'].astype(str)+ '<br> Percapita Income : ' + final_map['D_2014'].astype(str) + \
                '<br> Unemp Rate : ' + final_map['Rate'].astype(str)

scale = .05
cities = []
for j,i in enumerate(limits):
    df_sub = final_map[final_map['Organization Type']==i]
    #print i, ' : ',df_sub.count()
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        #locations = df_sub['code'],
        
        text = df_sub['text'],
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        #text = final_map['text'],
        
        marker = dict( 
            size = 8, 
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            #color = colors[j],
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            #color = scl[i],
            cmax = df_sub['prob0'].max(),
            )
        
        ,name = i)
   
    cities.append(city)
invest_types = []  
invst = final_map['Investment Type'].unique()

for j,i in enumerate(invst):
    df_sub = final_map[final_map['Investment Type']==i]
    #print i, ' : ',df_sub.count()
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        #locations = df_sub['code'],
        
        text = df_sub['text'],
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        #text = final_map['text'],
        
        marker = dict( 
            size = 8, 
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            cmax = df_sub['prob0'].max(),

            )
        ,name = i)
   
    invest_types.append(city)
    
layout = dict(
        title = ' By Organization Type And Investment Type',
         showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            showlakes = True,
            landcolor = 'rgb(217, 217, 217)',
            lakecolor = 'rgb(255, 255, 255)',
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
#fig = dict( data=invest_types, layout=layout )
fig = dict( data=cities+invest_types, layout=layout )
py.iplot( fig, validate=False, filename=' Contributions by County' )
#plot_url = py.plot(fig)